In [ ]:
from smolagents import CodeAgent, InferenceClientModel, Tool, tool
from dotenv import load_dotenv
import os
from langfuse import get_client
load_dotenv()
api_key = os.getenv('ANGFUSE_SECRET_KEY')

/home/dsi/yefimnu/miniconda3/envs/meal_planner/lib/python3.14/site-packages/langfuse/api/core/pydantic_utilities.py:10: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  import pydantic.v1 as pydantic_v1  # type: ignore  # nopycln: import


ConfigError: unable to infer type for attribute "description"